<a href="https://colab.research.google.com/github/rifat01-rahman/Project-of-Python/blob/main/Fill_the_missing_Values_for_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In the real world, We have missing values in time series data even though data is very much importatnt to conduct efficient analysis. We can not substract otherwise It will face shortage of the data. To resolve this issue we take some methods to fill the missing values. However, wrong techniques of fill missing values leads unreliable analysis. That's why this note book will help in this path.**

In [1]:
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.structural import UnobservedComponents
import pandas as pd

In [2]:
df_s = pd.read_excel('/content/East Asia and Pecific.xlsx')

In [3]:
df_s.head()

,Date,gdp,energy,co2
0,1985,3023.841209,862.567950,3774.2839
1,1986,3123.578525,880.440269,3913.0310
2,1987,3266.112050,901.246281,4093.4059
3,1988,3458.338258,938.669821,4393.6326
4,1989,3579.935959,956.525841,4556.8345


In [4]:
print(df_s.columns)

Index(['Date ', 'gdp', 'energy', 'co2'], dtype='object')


In [5]:
print(type(df_s))

<class 'pandas.core.frame.DataFrame'>


In [6]:
df_s.head()

,Date,gdp,energy,co2
0,1985,3023.841209,862.567950,3774.2839
1,1986,3123.578525,880.440269,3913.0310
2,1987,3266.112050,901.246281,4093.4059
3,1988,3458.338258,938.669821,4393.6326
4,1989,3579.935959,956.525841,4556.8345


In [7]:
df_s.columns = df_s.columns.str.strip() # WE have to remove space and some time we have to need to lower the words

In [8]:
df_a = df_s.copy()

In [9]:
# convert 'Date' column (years) to datetime
df_a['Date'] = pd.to_datetime(df_a['Date'], format='%Y')

# set as index
df_a = df_a.set_index('Date').sort_index()

In [10]:
df_a.head()

,gdp,energy,co2
Date,,,
1985-01-01,3023.841209,862.567950,3774.2839
1986-01-01,3123.578525,880.440269,3913.0310
1987-01-01,3266.112050,901.246281,4093.4059
1988-01-01,3458.338258,938.669821,4393.6326
1989-01-01,3579.935959,956.525841,4556.8345


In [11]:
print(type(df_a))

<class 'pandas.core.frame.DataFrame'>


**There are so many ways I have used here after that I show the graph because to see how actually all techniques dealing with missing values. To fill the missing values in time series is very much sensitive. We should care about it.
Obviously, Missing value range should not be large**

Different methods use in different circumstances as:
* polynomial methods we use, when we have polynomial features, for example- Business cycle.
* linear and time interpolation only use when data only follow linear pattern.
* Forward, Backward and rolling mean use when there is no jump, trend, seasonality doesn't work much.
* ARIMA methods deal with trends pattern, seasonality and some uncertainity as well, It's as sophesticated method.
* KNN impoter- Only use when we will see missing values has similiar behaviour as previous data or the nearest neighbours.





In [13]:
# ==============================================================
# Time Series Missing Value Imputation Toolkit
# Author: [Your Name]
# Purpose: Demonstrate multiple imputation techniques for time series
# ==============================================================

import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.structural import UnobservedComponents
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, kpss
from sklearn.impute import KNNImputer
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

# --------------------------------------------------------------
# 1. Setup: Assume df_s has a time series index (e.g., yearly, monthly)
# --------------------------------------------------------------
# Example setup if you have a "year" column:
# df_s['year'] = pd.to_datetime(df_s['year'], format='%Y')
# df_s = df_s.set_index('year').sort_index()

assert 'energy' in df_a.columns, "DataFrame must contain a 'energy' column"

# --------------------------------------------------------------
# 2. Simple Deterministic Methods
# --------------------------------------------------------------
df_a['linear_interp'] = df_a['energy'].interpolate(method='linear')
df_a['time_interp']   = df_a['energy'].interpolate(method='time')
df_a['ffill']         = df_a['energy'].ffill()
df_a['bfill']         = df_a['energy'].bfill()
df_a['rolling_mean']  = df_a['energy'].fillna(df_a['energy'].rolling(window=3, min_periods=1).mean())
df_a['poly_interp']   = df_a['energy'].interpolate(method='polynomial', order=2)
df_a['spline_interp'] = df_a['energy'].interpolate(method='spline', order=2)

# --------------------------------------------------------------
# 3. Statistical Model-Based Methods
# --------------------------------------------------------------

# --- Kalman Filter (local level model) ---
mod = UnobservedComponents(df_a['energy'], level='local level')
res = mod.fit(disp=False)
df_a['kalman'] = res.filter_results.smoothed_state[0]

# Step 0: Copy series
y = df_a['energy'].copy()
y_imputed = y.copy()
y_nonan = y.dropna()

# Step 1: Decide differencing order
def choose_d(y):
    adf_p = adfuller(y)[1]
    try:
        kpss_p = kpss(y, regression='c', nlags="auto")[1]
    except:
        kpss_p = 0.01
    d = 0 if adf_p < 0.05 and kpss_p > 0.05 else 1
    print(f"ADF p={adf_p:.3f}, KPSS p={kpss_p:.3f} → d={d}")
    return d

d = choose_d(y_nonan)

# Step 2: Grid search for (p,q)
best_aic = np.inf
best_order = (0,d,0)
for p in range(6):
    for q in range(6):
        try:
            model = ARIMA(y_nonan, order=(p,d,q))
            results = model.fit()
            if results.aic < best_aic:
                best_aic = results.aic
                best_order = (p,d,q)
        except:
            continue

print("Selected ARIMA order:", best_order, "with AIC:", best_aic)

# Step 3: Impute missing values iteratively
for i in range(len(y_imputed)):
    if pd.isna(y_imputed.iloc[i]):
        temp_y = y_imputed.iloc[:i].dropna()
        if len(temp_y) < max(2, best_order[0]+best_order[2]):
            # Not enough data to fit ARIMA, use simple forward fill
            y_imputed.iloc[i] = temp_y.iloc[-1] if len(temp_y)>0 else 0
        else:
            try:
                model = ARIMA(temp_y, order=best_order)
                fit = model.fit()
                y_imputed.iloc[i] = fit.forecast(steps=1)[0]
            except:
                y_imputed.iloc[i] = temp_y.iloc[-1]  # fallback

df_a['arima'] = y_imputed

# --------------------------------------------------------------
# 4. Machine Learning-Based Method
# --------------------------------------------------------------
knn_imp = KNNImputer(n_neighbors=3)
df_a['knn'] = knn_imp.fit_transform(df_a[['energy']])[:,0]

# --------------------------------------------------------------
# 5. Visualization (Plotly Interactive)
# --------------------------------------------------------------
fig = go.Figure()

# Original series
fig.add_trace(go.Scatter(
    x=df_a.index,
    y=df_a['energy'],
    mode='lines+markers',
    name='Original (with NaN)',
    line=dict(color='black', dash='dash'),
    marker=dict(symbol='circle')
))

methods = [
    'linear_interp', 'time_interp', 'ffill', 'bfill',
    'rolling_mean', 'poly_interp', 'spline_interp',
    'kalman', 'arima', 'knn'
]
colors = [
    'blue', 'orange', 'green', 'red',
    'purple', 'brown', 'pink',
    'gray', 'gold', 'teal'
]

for method, color in zip(methods, colors):
    fig.add_trace(go.Scatter(
        x=df_a.index,
        y=df_a[method],
        mode='lines+markers',
        name=method,
        line=dict(color=color),
        marker=dict(symbol='circle')
    ))

fig.update_layout(
    title="Comparison of Missing Value Imputation Methods",
    xaxis_title="Time",
    yaxis_title="Value",
    hovermode="x unified",
    template="plotly_white",
    width=1000,
    height=600
)

fig.show()



ADF p=0.992, KPSS p=0.010 → d=1
Selected ARIMA order: (1, 1, 2) with AIC: 362.9270942743715


In [ ]:
df_s

,country,gdp,edu,energy,carbon,linear_interp,time_interp,ffill,bfill,rolling_mean,poly_interp,spline_interp,kalman,arima,knn
data,,,,,,,,,,,,,,,
1985-01-01,3,848.459509,NaN,328.485757,43.9230,NaN,NaN,NaN,4.075070,NaN,NaN,NaN,4.075238,NaN,7.290373
1986-01-01,3,865.383780,4.075070,331.404492,47.1303,4.075070,4.075070,4.075070,4.075070,4.075070,4.075070,4.075070,4.075241,4.075070,4.075070
1987-01-01,3,890.342223,4.057950,351.897714,54.5186,4.057950,4.057950,4.057950,4.057950,4.057950,4.057950,4.057950,4.078725,4.057950,4.057950
1988-01-01,3,926.557789,4.535500,357.186721,58.7068,4.535500,4.535500,4.535500,4.535500,4.535500,4.535500,4.535500,4.504762,4.535500,4.535500
1989-01-01,3,941.244309,NaN,365.169138,63.4486,4.327925,4.327641,4.535500,4.120350,4.296725,4.516956,3.736770,4.305600,4.501272,7.290373
1990-01-01,3,950.884645,4.120350,370.756577,65.6223,4.120350,4.120350,4.120350,4.120350,4.120350,4.120350,4.120350,4.106439,4.120350,4.120350
1991-01-01,3,966.522496,NaN,367.121772,68.1079,3.620460,3.620460,4.120350,3.120570,4.120350,3.627888,3.574474,3.624324,4.160309,7.290373
1992-01-01,3,1011.625084,3.120570,379.579214,74.8929,3.120570,3.120570,3.120570,3.120570,3.120570,3.120570,3.120570,3.142209,3.120570,3.120570
1993-01-01,3,1001.258548,NaN,387.504828,83.2756,3.077261,3.077175,3.120570,2.644170,3.120570,2.678277,3.501091,3.100232,3.220631,7.290373
